# Исследование надёжности заёмщиков #

Рассмотрим в этом проекте как влияют различные признаки на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

**Цель исследования** — проверить влияет ли семейное положение и количество детей на вероятность возврата кредита в срок


# Обзор данных #

In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
try:
    df = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в df
except:
    df = pd.read_csv('C:\\Users\\User\\Downloads\\data_3_sprint.csv')

In [3]:
display(df.head(10)) # просмотр первых 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице одиннадцать столбцов. В данных присутствует 3 типа данных — `object`, `int64`, `float64`.

Согласно документации к данным:

* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type`— тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

В названиях колонок нет нарушений стиля.

Общее количество строк в данных 21525. В столбцах `days_employed` и `total_income` есть пропущенные значения.
<br>При предварительном просмотре сразу видны отрицательные значения в колонке days_employed и слишком большие значения, которые не могут быть днями.

### Шаг 2.1 Заполнение пропусков

In [5]:
print(df.isna().sum()) # находим количество пропущенных значений

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [6]:
display(df[df['days_employed'].isna()].head()) # находим строки с пропущенными значениями и выводим на экран первые пять

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [7]:
days_employed_share = len(df[df['days_employed'].isna()]) / df.shape[0] 
# находим долю отсутствующих значений в столбце days_employed

In [8]:
total_income_share = len(df[df['total_income'].isna()]) / df.shape[0] 
# находим долю отсутствующих значений в столбце total_income

In [9]:
print('В столбце days_employed значения отсутствуют в: {:.1%} случаев.' .format(days_employed_share))
print('В столбце total_income значения отсутствуют в: {:.1%} случаев.' .format(total_income_share))

В столбце days_employed значения отсутствуют в: 10.1% случаев.
В столбце total_income значения отсутствуют в: 10.1% случаев.


Перед тем как работать с пропущенными значениями необходимо избавиться от аномалий в данных. В столбце `days_employed` заменить отрицательные значения положительными. 

In [10]:
df['days_employed'] = df['days_employed'].abs() # заменяем отрицательные значения столбца days_employed на положительные

В столбце days_employed так же есть аномальные значения трудового стажа. Но эти данные в расчетах не пригодятся. 

In [11]:
print('Количество человек имеющие стаж больше 800 лет:', df[df['days_employed'] > 300000]['days_employed'].count())

Количество человек имеющие стаж больше 800 лет: 3445


In [12]:
df['total_income'] =  df.total_income.fillna(df.groupby('income_type')['total_income'].transform('median')) 
# замена пропусков в total_income медианным значением по типу занятости 

In [13]:
df['days_employed'] =  df.total_income.fillna(df.groupby('income_type')['days_employed'].transform('median')) 
# замена пропусков в days_employed медианным значением по типу занятости

In [20]:
df.info() # проверяем что пропущенных данных не осталось

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Ежемесячный доход и общий трудовой стаж в днях количественные переменные. Для заполнения пропусков в количественных переменных используют значения характеризующие состояние выборки. Чтобы найти типичные значения можно применить среднее арифметическое или медиану. В данном случае медиану использовать предпочтительнее, т.к. в данных есть значения сильно отличающиеся от большинства. 

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [21]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Скорее всего в столбце с количеством детей допущены ошибки. Заменяем количество детей с двадцати на два и с минус один на один.

In [22]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)

In [23]:
df['gender'].value_counts() # 

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Скорее всего XNA это пропущенное значение. В расчетах не пригодится. 

### Шаг 2.3. Изменение типов данных.

In [24]:
df['total_income'] = df['total_income'].astype('int') # замена типа данных на целочисленный

### Шаг 2.4. Удаление дубликатов.

In [25]:
print(df.duplicated().sum()) # находим количество дубликатов

54


In [26]:
duplicated_df = df[df.duplicated()].head() # датафрейм с дубликатами
display(duplicated_df) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,142594.396847,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,142594.396847,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,118514.486412,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,118514.486412,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,118514.486412,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы


In [27]:
df = df.drop_duplicates().reset_index() # удаление дубликатов

In [28]:
df['education'].value_counts() # подсчет уникальных значений для поиска неявных дубликатов

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Разные названия с одинаковым значением. Приведём к одному регистру.

In [29]:
df['education'] = df['education'].str.lower() # приведение в нижний регистр столбца education

In [30]:
df['education'].value_counts() # повторная проверка на неявные дубликаты

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [31]:
df['education_id'].value_counts() # проверка на неявные дубликаты

1    15188
0     5251
2      744
3      282
4        6
Name: education_id, dtype: int64

In [32]:
df['family_status'].value_counts() # проверка на неявные дубликаты

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [33]:
df['family_status_id'].value_counts() # проверка на неявные дубликаты

0    12344
1     4163
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

In [34]:
df['income_type'].value_counts() # проверка на неявные дубликаты

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [35]:
df['purpose'].value_counts() # проверка на неявные дубликаты

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Много неявных дубликатов с целями. Требуется применить категоризацию.

Явные дубликаты удалили с помощью метода drop_duplicates(). Этот метод находит явные дубликаты и удаляет их. 
<br> Неявные дубликаты необходимо искать вручную. Это удобно делать с помощью метода value_counts(). Этот метод выводит на <br>экран повторяющиеся значения с количеством их повторений.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [36]:
education_df = df[['education_id','education']] # создание датафрейма с колонками education_id, education

In [37]:
education_df = education_df.drop_duplicates().reset_index() # удаление дубликатов

In [38]:
display(education_df)

,index,education_id,education
0,0,0,высшее
1,1,1,среднее
2,13,2,неоконченное высшее
3,31,3,начальное
4,2962,4,ученая степень


In [39]:
# создание датафрейма с колонками family_status_id, family_status
family_status_df = df[['family_status_id', 'family_status']] 

In [40]:
family_status_df = family_status_df.drop_duplicates().reset_index() 

In [41]:
display(family_status_df)

,index,family_status_id,family_status
0,0,0,женат / замужем
1,4,1,гражданский брак
2,18,2,вдовец / вдова
3,19,3,в разводе
4,24,4,Не женат / не замужем


In [42]:
df.drop(['family_status', 'education'], axis=1) # удаление столбцов family_status и education

,index,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,253875.639453,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,1,112080.014102,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,2,0,145885.952297,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,3,267628.550329,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,4,0,158616.077870,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...
21466,21520,1,224791.862382,43,1,1,F,компаньон,0,224791,операции с жильем
21467,21521,0,155999.806512,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21468,21522,1,89672.561153,38,1,1,M,сотрудник,1,89672,недвижимость
21469,21523,3,244093.050500,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.6. Категоризация дохода.

In [43]:
def total_income_category(row):                      # создание функции для создания нового столбца total_income_category
    total_income = row['total_income']
    if 0 <= total_income <= 30000:
        return 'E'        
    if 30001 <= total_income <= 50000:
        return 'D'        
    if 50001 <= total_income <= 200000:
        return 'C'  
    if 200001 <= total_income <= 1000000:
        return 'B'  
    return 'A'

df['total_income_category'] = df.apply(total_income_category, axis=1)
display(df.head(10))

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,0,1,253875.639453,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,1,112080.014102,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,2,0,145885.952297,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,3,267628.550329,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,4,0,158616.077870,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,5,0,255763.565419,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B
6,6,0,240525.971920,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B
7,7,0,135823.934197,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C
8,8,2,95856.832424,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,9,0,144425.938277,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C


In [44]:
df['total_income_category'].value_counts() # проверяем не осталось ли строк без категории

C    16032
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

In [45]:
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [46]:
def purpose_category(row):    # создание функции для создания нового столбца purpose_category
    purpose = row['purpose']
    if 'автом' in purpose:
        return 'Операции с автомобилем'        
    if 'недвиж' in purpose:
        return 'Операции с недвижимостью'
    if 'жиль' in purpose:
        return 'Операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'Проведение свадьбы'  
    if 'образов' in purpose:
        return 'Получение образования'  
    
df['purpose_category'] = df.apply(purpose_category, axis=1)
display(df.head(10))

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,0,1,253875.639453,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,Операции с недвижимостью
1,1,1,112080.014102,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,Операции с автомобилем
2,2,0,145885.952297,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,Операции с недвижимостью
3,3,3,267628.550329,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,Получение образования
4,4,0,158616.077870,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,Проведение свадьбы
5,5,0,255763.565419,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,Операции с недвижимостью
6,6,0,240525.971920,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,Операции с недвижимостью
7,7,0,135823.934197,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,Получение образования
8,8,2,95856.832424,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,Проведение свадьбы
9,9,0,144425.938277,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,Операции с недвижимостью


In [47]:
df['purpose_category'].value_counts() # проверяем не осталось ли строк без категории

Операции с недвижимостью    10814
Операции с автомобилем       4308
Получение образования        4014
Проведение свадьбы           2335
Name: purpose_category, dtype: int64

### Шаг 3. Рассмотрим какие признаки могут влиять на возврат кредита в срок

#### Вопрос 1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [48]:
df_pivot_children = df.pivot_table(index=['debt'], columns='children', values = 'family_status_id', aggfunc='count')
print(df_pivot_children)

children        0       1       2      3     4    5
debt                                               
0         13044.0  4411.0  1926.0  303.0  37.0  9.0
1          1063.0   445.0   202.0   27.0   4.0  NaN


In [49]:
ch0 = df_pivot_children[0][1] / (df_pivot_children[0][0] + df_pivot_children[0][1])
ch1 = df_pivot_children[1][1] / (df_pivot_children[1][0] + df_pivot_children[1][1])
ch2 = df_pivot_children[2][1] / (df_pivot_children[2][0] + df_pivot_children[2][1])
ch3 = df_pivot_children[3][1] / (df_pivot_children[3][0] + df_pivot_children[3][1])
ch4 = df_pivot_children[4][1] / (df_pivot_children[4][0] + df_pivot_children[4][1])

print("{0:.2f}% Нет детей".format(ch0*100))
print("{0:.2f}% 1 Ребенок".format(ch1*100))
print("{0:.2f}% 2 Ребенка".format(ch2*100))
print("{0:.2f}% 3 Ребенка".format(ch3*100))
print("{0:.2f}% 4 Ребенка".format(ch4*100))

7.54% Нет детей
9.16% 1 Ребенок
9.49% 2 Ребенка
8.18% 3 Ребенка
9.76% 4 Ребенка


С увеличением количества детей растет вероятность невыплаты кредита в срок. Хотя люди с тремя детьми выплачивают кредит в срок чаще, чем люди с одним ребенком, но все равно реже, чем люди без детей.

#### Вопрос 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [50]:
df_family_status = df.pivot_table(index=['family_status'], columns='debt', values='family_status_id', aggfunc='count')
print(df_family_status)

debt                       0    1
family_status                    
Не женат / не замужем   2536  274
в разводе               1110   85
вдовец / вдова           896   63
гражданский брак        3775  388
женат / замужем        11413  931


In [51]:
no_family = df_family_status[1][0] / (df_family_status[0][0] + df_family_status[1][0])
divorce = df_family_status[1][1] / (df_family_status[0][1] + df_family_status[1][1])
widower = df_family_status[1][2] / (df_family_status[0][2] + df_family_status[1][2])
partner = df_family_status[1][3] / (df_family_status[0][3] + df_family_status[1][3])
family = df_family_status[1][4] / (df_family_status[0][4] + df_family_status[1][4])
print("{0:.2f}% Не женат / не замужем".format(no_family*100))
print("{0:.2f}% гражданский брак".format(partner*100))
print("{0:.2f}% женат / замужем".format(family*100))
print("{0:.2f}% в разводе".format(divorce*100))
print("{0:.2f}% вдовец / вдова".format(widower*100))

9.75% Не женат / не замужем
9.32% гражданский брак
7.54% женат / замужем
7.11% в разводе
6.57% вдовец / вдова


Есть зависимость между семейным положением и возвратом кредита в срок. Люди не в браке или в гражданском браке реже выплачивают кредит в срок. 

#### Вопрос 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [52]:
df_pivot_total_income = df.pivot_table(index=['total_income_category'], columns='debt', values='index', aggfunc='count')
print(df_pivot_total_income) 

debt                       0     1
total_income_category             
A                         23     2
B                       4686   356
C                      14672  1360
D                        329    21
E                         20     2


In [53]:
a_category = df_pivot_total_income[1][0] / (df_pivot_total_income[0][0] + df_pivot_total_income[1][0])
b_category = df_pivot_total_income[1][1] / (df_pivot_total_income[0][1] + df_pivot_total_income[1][1])
c_category = df_pivot_total_income[1][2] / (df_pivot_total_income[0][2] + df_pivot_total_income[1][2])
d_category = df_pivot_total_income[1][3] / (df_pivot_total_income[0][3] + df_pivot_total_income[1][3])
e_category = df_pivot_total_income[1][4] / (df_pivot_total_income[0][4] + df_pivot_total_income[1][4])
print("У {0:.2f}% заработок до 30000".format(e_category*100))
print("У {0:.2f}% заработок больше 50001 и меньше 200000".format(c_category*100))
print("У {0:.2f}% заработок больше 1000000".format(a_category*100))
print("У {0:.2f}% заработок больше 200001 и меньше 1000000".format(b_category*100))
print("У {0:.2f}% заработок больше 30001 и меньше 50000".format(d_category*100))

У 9.09% заработок до 30000
У 8.48% заработок больше 50001 и меньше 200000
У 8.00% заработок больше 1000000
У 7.06% заработок больше 200001 и меньше 1000000
У 6.00% заработок больше 30001 и меньше 50000


Люди с наименьшим доходом реже выплачивают кредит в срок. В целом явной зависимости скоре нет. Однако люди с заработком больше 30001 и меньше 50000 чаще выплачивают кредит в срок, а люди с заработком до 30000 реже выплачивают кредит в срок.

#### Вопрос 4. Как разные цели кредита влияют на его возврат в срок?

In [54]:
df_pivot_purpose = df.pivot_table(index=['purpose_category'], columns='debt', values='index', aggfunc='count')
print(df_pivot_purpose) 

debt                          0    1
purpose_category                    
Операции с автомобилем     3905  403
Операции с недвижимостью  10032  782
Получение образования      3644  370
Проведение свадьбы         2149  186


In [55]:
car = df_pivot_purpose[1][0] / (df_pivot_purpose[0][0] + df_pivot_purpose[1][0])
estate = df_pivot_purpose[1][1] / (df_pivot_purpose[0][1] + df_pivot_purpose[1][1])
education = df_pivot_purpose[1][2] / (df_pivot_purpose[0][2] + df_pivot_purpose[1][2])
wedding = df_pivot_purpose[1][3] / (df_pivot_purpose[0][3] + df_pivot_purpose[1][3])
print("{0:.2f}% автомобиль".format(car*100))
print("{0:.2f}% недвижимость".format(estate*100))
print("{0:.2f}% образование".format(education*100))
print("{0:.2f}% свадьба".format(wedding*100))

9.35% автомобиль
7.23% недвижимость
9.22% образование
7.97% свадьба


Чаще в срок платят кредит на свадьбу и недвижимсть. Просрочка чаще по кредитам на образование и покупку автомобиля. Возможно из-за того, что к покупке недвижимости люди подходят серьезнее, потому что срок и сумма кредита обычно намного больше, чем на другие цели.

## Общий вывод:

После проведенного исследования можно сказать, что семейное положение и количество детей влияют на возврат кредита в срок. 
С увеличением количества детей растет вероятность невыплаты кредита в срок. Также люди не бывавшие в браке реже выплачивают кредит в срок.
<br>Более добросовестными заемщиками являются вдовцы или разведенные люди. И люди без детей.